### ImpactsInYear

In [0]:
select Year(impact.Impact_date) as Year, MONTH(impact.Impact_date) as Month,impact.Severity, COUNT(*) as Impact_Count
from NA_PDC_POWERFLEET.impact as impact
where impact.Operator <> 'Unassigned' and impact.Opr_Class <> 'Maintenance'
and  impact.Vehicle not like 'TT%'
group by Year(impact.Impact_date),MONTH(impact.Impact_date), impact.Severity
having 
Year(impact.Impact_date) = '{{ Impact_Year }}'
and impact.Severity in ( {{ Severity }} )
order by Year,Month asc

File "<command-3900503903318157>" , line 1 
 select Year(impact.Impact_date) as Year, MONTH(impact.Impact_date) as Month,impact.Severity, COUNT(*) as Impact_Count 
 ^
 SyntaxError : invalid syntax

### ImpactsInMonth

In [0]:
select MONTH(impact.Impact_date) as Month,DAY(impact.Impact_date) as Day, impact.Severity, COUNT(*) as Impact_Count
from NA_PDC_POWERFLEET.impact as impact
join NA_PDC_POWERFLEET.operator as operator on (upper(impact.First_Name) = operator.UP_CASE_FRST_NM and upper(impact.Last_Name) = operator.UP_CASE_LAST_NM)
where Year(impact.Impact_date) = {{Impact_Year}}
and impact.Operator <> 'Unassigned' and impact.Opr_Class <> 'Maintenance'
and  impact.Vehicle not like 'TT%'
group by MONTH(impact.Impact_date),DAY(impact.Impact_date), impact.Severity
having 
MONTH(impact.Impact_date) = {{ Impact_Month }} 
and impact.Severity in ( {{ Severity }} )
order by Day asc

File "<command-3900503903318165>" , line 1 
 select impact.Operator ,impact.Severity, count(*) as impact_count 
 ^
 SyntaxError : invalid syntax

### ImpactsByOperator

In [0]:

select impact.Operator ,impact.Severity, count(*) as impact_count
from NA_PDC_POWERFLEET.impact impact
where impact.impact_Date >= '{{Impact_StartDate}}' and impact.impact_Date < '{{Impact_EndDate}}'
and impact.Operator <> 'Unassigned' and impact.Opr_Class <> 'Maintenance'
and  impact.Vehicle not like 'TT%'
group by impact.Severity,impact.Operator
having impact.Severity in ( {{ Severity }} )
order by impact_count desc;


#### ImpactsByVehicle

In [0]:
select  Vehicle.Vehicle,impact.Severity, count(*) as impact_count
from NA_PDC_POWERFLEET.impact impact, NA_PDC_POWERFLEET.Vehicle  Vehicle
where impact.Vehicle = Vehicle.Vehicle and Vehicle.`In Service` = 'Yes'
 and impact.impact_Date >= '{{Impact_StartDate}}' and impact.impact_Date < '{{Impact_EndDate}}'
and impact.Operator <> 'Unassigned' and impact.Opr_Class <> 'Maintenance'
and  impact.Vehicle not like 'TT%'
group by impact.Severity, Vehicle.Vehicle
Having  impact.Severity in ( {{ Severity }} )
order by impact_count desc

### ImpactsByShift

In [0]:
select TBL_DAYTIME.DAYTIME,Severity, count(A.impact_id) as impact_count
from NA_PDC_POWERFLEET.Impact A,
(select Impact_date as IMAPACT_TIME,
  CASE WHEN HOUR(Impact_date) >= 6
     AND HOUR(Impact_date) <= 11 THEN 'MORNING'
    WHEN HOUR(Impact_date) >= 12
     AND HOUR(Impact_date) <= 17 THEN 'AFTER NOON'
    WHEN HOUR(Impact_date) >= 18
     AND HOUR(Impact_date) <= 19 THEN 'EVENING'
    ELSE 'NIGHT'
  END AS DAYTIME
FROM
  NA_PDC_POWERFLEET.Impact) as TBL_DAYTIME
  where A.impact_date = TBL_DAYTIME.IMAPACT_TIME
  and Operator <> 'Unassigned' and Opr_Class <> 'Maintenance'
and  Vehicle not like 'TT%'
  and impact_Date >= '{{Impact_StartDate}}' and impact_Date < '{{Impact_EndDate}}'
  and Severity in ( {{ Severity }} )
  group by TBL_DAYTIME.DAYTIME, Severity
  order by count(A.impact_id) desc

### ImapctsByOperatorSeniority

In [0]:
select operator.SENIORITY,impact.Severity, count(*) as impact_count,
CASE WHEN TRIM(operator.SENIORITY) = '6- MON'
	     THEN 0
	   WHEN TRIM(operator.SENIORITY) = '6+ MON'
	     THEN 0.5
	   WHEN TRIM(operator.SENIORITY) = '1+ YR'
	    THEN 1
	   WHEN TRIM(operator.SENIORITY) = '5+ YR'
	    THEN 5
	   WHEN TRIM(operator.SENIORITY) = '10+ YR'
	    THEN 10
	   WHEN TRIM(operator.SENIORITY) = '15+ YR'
	    THEN 15
	   WHEN TRIM(operator.SENIORITY) = '20+ YR'
	    THEN 20
	   ELSE 30
	  END AS SENIORITY_YEAR
from NA_PDC_POWERFLEET.impact impact
join na_pdc_powerfleet.operator
where impact.impact_Date >= '{{Impact_StartDate}}' and impact.impact_Date < '{{Impact_EndDate}}'
and impact.Operator <> 'Unassigned' and impact.Opr_Class <> 'Maintenance'
and  impact.Vehicle not like 'TT%'
group by impact.Severity, operator.SENIORITY
having impact.Severity in ( {{ Severity }} )
order by SENIORITY_YEAR ASC;





### UsageInYear

In [0]:
select Year(usage.login),month(usage.login), sum(usage.Login_hrs),sum(usage.activity_hrs)
from NA_PDC_POWERFLEET.usage as usage
where (usage.operator <> '[Unassigned]' and usage.opr_class <> 'Maintenance')
AND usage.Vehicle not like 'TT%'
group by Year(usage.login),month(usage.login)
having Year(usage.login) = {{Usage_Year }};



#### UsageInMonth

In [0]:
select Day(usage.login) as Day, sum(usage.Login_hrs),sum(usage.activity_hrs)
from NA_PDC_POWERFLEET.usage as usage
where Year(usage.login) = {{ Usage_Year }}
and (usage.operator <> '[Unassigned]' and usage.opr_class <> 'Maintenance')
AND usage.Vehicle not like 'TT%'
and month(usage.login) = {{Usage_Month}}
group by Day(usage.login)
order by Day asc


### UsageByOperator

In [0]:
SELECT usage.Operator, sum(usage.Login_hrs) as total_login_hrs, sum(usage.Activity_Hrs) as total_activity_hrs,sum(Total_Impacts) as total_impacts
FROM na_pdc_powerfleet.usage
where usage.Login >= '{{Usage_StartDate}}' and usage.Login < '{{Usage_EndDate}}'
and usage.operator <> '[Unassigned]' and  usage.opr_class <> 'Maintenance'
AND usage.Vehicle not like 'TT%'
group by usage.Operator
order by total_activity_hrs desc, total_impacts desc

### UsageByVehicle

In [0]:
SELECT usage.Vehicle, sum(usage.Login_hrs) as total_login_hrs, sum(usage.Activity_Hrs) as total_activity_hrs,sum(Total_Impacts) as total_impacts
FROM na_pdc_powerfleet.usage
where usage.Login >= '{{Usage_StartDate}}' and usage.Login < '{{Usage_EndDate}}'
and usage.operator <> '[Unassigned]' and usage.operator not like '%Maintenance%'
and usage.Vehicle not like 'TT%'
group by usage.Vehicle
order by total_activity_hrs desc, total_impacts desc

### UsageByOperatorVehicle

In [0]:
SELECT usage.Operator,usage.Vehicle, sum(usage.Activity_Hrs) as total_activity_hrs,sum(Total_Impacts) as total_impacts
FROM na_pdc_powerfleet.usage
where usage.Login >= '{{Usage_StartDate}}' and usage.Login < '{{Usage_EndDate}}'
and (usage.operator <> '[Unassigned]' and usage.opr_class <> 'Maintenance')
AND usage.Vehicle not like 'TT%'
group by usage.Vehicle, usage.Operator
order by total_activity_hrs desc, total_impacts desc

### UnAssignedVehiclesEngineOffHours

In [0]:

SELECT usage.Vehicle, sum(usage.Eng_Off_hrs)
FROM na_pdc_powerfleet.usage usage
WHERE usage.Assigned = false
and usage.Login >= '{{Usage_StartDate}}' and usage.Login < '{{Usage_EndDate}}'
GROUP BY usage.Vehicle

### OperatorUsageActivityPercentage

In [0]:
		SELECT usage.operator, sum(usage.Login_hrs)/sum(usage.Activity_Hrs) as percent_activity_hrs
	FROM na_pdc_powerfleet.usage usage
	where usage.Login >= '{{Usage_StartDate}}' and usage.Login < '{{Usage_EndDate}}'
	and (usage.operator <> '[Unassigned]' and usage.opr_class <> 'Maintenance')
AND usage.Vehicle not like 'TT%'
	group by usage.operator
	having percent_activity_hrs is not null
	order by percent_activity_hrs desc
	

### VehicleCheckListSevCode

In [0]:
select checklist.Vehicle, checklist.Sev_Code, count(*) as Severity_count
from NA_PDC_POWERFLEET.checklist as checklist
where checklist.DateCompleted >= '{{Usage_StartDate}}' and checklist.DateCompleted < '{{Usage_EndDate}}'
and checklist.Vehicle not like 'TT%'
group by checklist.Vehicle, checklist.Sev_Code
having checklist.Sev_Code in ( {{Checklist_Severity }} )
order by Severity_count desc;


### CheckListSevCodeWithinIn2DaysOfImpact

In [0]:
select checklist.Vehicle,checklist.Sev_Code, count(*) as count_sev_codes
from na_pdc_powerfleet.checklist checklist
join 
(
   select impact.Vehicle,impact.Impact_Date
   from na_pdc_powerfleet.impact impact
   where impact.Severity in ( {{ Severity }} )
   and impact.impact_Date >= '{{Impact_StartDate}}' and impact.impact_Date < '{{Impact_EndDate}}'
) impact_vehicle on (impact_vehicle.Vehicle = checklist.Vehicle)
and checklist.DateCompleted >= date_sub(to_date(impact_vehicle.Impact_Date), 2) and checklist.DateCompleted < to_date(impact_vehicle.Impact_Date)
where checklist.Sev_Code in ({{Checklist_Severity}})
group by checklist.Vehicle,checklist.Sev_Code
order by count_sev_codes desc

### SleeperByVehicle

In [0]:
select sleeper.Vehicle,sleeper.operator,sleeper.Opr_class, sum(sleeper.Sleep_Time_Mins)/60 as SleepOverage_hrs
from na_pdc_powerfleet.sleeper as sleeper
where sleeper.Sleep_Time_Mins > sleeper.Sleep_Thresh_Mins
and sleeper.Sleep_Start >= '{{Sleep_Start}}'  and sleeper.Sleep_End < '{{Sleep_End}}'
and sleeper.operator <> '[Unassigned]' and sleeper.Opr_Class <> 'Maintenance'
and sleeper.Vehicle not like 'TT%'
group by sleeper.Vehicle,sleeper.operator,sleeper.Opr_class
order by SleepOverage_hrs desc

### SleeperByOperator

In [0]:
select sleeper.Operator, sleeper.opr_class, sum(sleeper.Sleep_Time_Mins - sleeper.Sleep_Thresh_Mins)/60 as SleepOverage_hrs
from na_pdc_powerfleet.sleeper as sleeper
where sleeper.Sleep_Time_Mins > sleeper.Sleep_Thresh_Mins
and sleeper.Sleep_Start >= '{{Sleep_Start}}'  and sleeper.Sleep_End < '{{Sleep_End}}'
and sleeper.operator <> '[Unassigned]' and sleeper.Opr_Class <> 'Maintenance'
and sleeper.Vehicle not like 'TT%'
group by sleeper.Operator,sleeper.opr_class
order by SleepOverage_hrs desc